# Lab 6-1: Softmax Classification

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    We use elemental PyTorch to implement linear regression here. However, in most actual applications, abstractions such as <code>nn.Module</code> or <code>nn.Linear</code> are used.
</div>

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Softmax

Convert numbers to probabilities with softmax.

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [80]:
z = torch.FloatTensor([[1, 2, 3],[4, 9, 6]])
print(z,z.shape)

tensor([[1., 2., 3.],
        [4., 9., 6.]]) torch.Size([2, 3])


PyTorch has a `softmax` function.

In [83]:
hypothesis = F.softmax(z, dim=1)  # dim 0 指的是每一列 dim 1 指的是每一行
# hypothesis_hand = torch.exp(z) / torch.sum(torch.exp(z)) # 手撸
print(hypothesis)
# print(hypothesis_hand)

predicate = torch.exp(torch.FloatTensor([9])) / (torch.exp(torch.FloatTensor([4])) + torch.exp(torch.FloatTensor([9])) + torch.exp(torch.FloatTensor([6])) )
print(predicate)


tensor([[0.0900, 0.2447, 0.6652],
        [0.0064, 0.9465, 0.0471]])
tensor([0.9465])


In [36]:
hypothesis.sum(dim=0)

tensor(1.)

## Cross Entropy Loss (Low-level)

For multi-class classification, we use the cross entropy loss.

$$ Loss = \frac{1}{N} \sum - y \log(\hat{y}) $$

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (0 or 1).

In [123]:
z = torch.rand(3, 5, requires_grad=True)
print(z,z.shape)
hypothesis = F.softmax(z, dim=1) # 将每一行做softmax处理
print(hypothesis,torch.sum(hypothesis,dim=1))

tensor([[0.6547, 0.3868, 0.6922, 0.6616, 0.8053],
        [0.8367, 0.3307, 0.9885, 0.4422, 0.4828],
        [0.0281, 0.1782, 0.2079, 0.2861, 0.8555]], requires_grad=True) torch.Size([3, 5])
tensor([[0.2011, 0.1538, 0.2088, 0.2025, 0.2338],
        [0.2414, 0.1455, 0.2810, 0.1627, 0.1694],
        [0.1441, 0.1674, 0.1725, 0.1865, 0.3296]], grad_fn=<SoftmaxBackward0>) tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


In [124]:
y = torch.randint(5, (3,)).long()
print(y,y.shape)

tensor([2, 0, 0]) torch.Size([3])


In [125]:
y_one_hot = torch.zeros_like(hypothesis) # 创建一个同等shape的矩阵
print(y_one_hot,y_one_hot.shape)
print(y,y.unsqueeze(1),y.unsqueeze(1).shape) # 输出为 [3] [4] [3]


y_one_hot.scatter_(1, y.unsqueeze(1), 1) # 例如，如果 y.unsqueeze(1) 的结果为 [[0], [2], [1]]，那么 y_one_hot 中的第0、第2、第1个维度（第二维度）将被设置为1，其余位置为0。
# scatter => (dim,target,value)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]) torch.Size([3, 5])
tensor([2, 0, 0]) tensor([[2],
        [0],
        [0]]) torch.Size([3, 1])


tensor([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [127]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6418, grad_fn=<MeanBackward0>)


## Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [128]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.6039, -1.8719, -1.5665, -1.5971, -1.4534],
        [-1.4214, -1.9274, -1.2696, -1.8159, -1.7753],
        [-1.9374, -1.7873, -1.7576, -1.6793, -1.1100]], grad_fn=<LogBackward0>)

In [129]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.6039, -1.8719, -1.5665, -1.5971, -1.4534],
        [-1.4214, -1.9274, -1.2696, -1.8159, -1.7753],
        [-1.9374, -1.7873, -1.7576, -1.6793, -1.1100]],
       grad_fn=<LogSoftmaxBackward0>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [130]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

print("(y_one_hot * -torch.log(F.softmax(z, dim=1)))\n",(y_one_hot * -torch.log(F.softmax(z, dim=1))))
print("\n(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1)\n",(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1))
print("\n(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()\n",(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean())


(y_one_hot * -torch.log(F.softmax(z, dim=1)))
 tensor([[0.0000, 0.0000, 1.5665, 0.0000, 0.0000],
        [1.4214, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.9374, 0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>)

(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1)
 tensor([1.5665, 1.4214, 1.9374], grad_fn=<SumBackward1>)

(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
 tensor(1.6418, grad_fn=<MeanBackward0>)


In [131]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.6418, grad_fn=<NllLossBackward0>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [132]:
F.cross_entropy(z, y) # 直接算loss

tensor(1.6418, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [133]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [134]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## Training with `F.cross_entropy`

In [17]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## High-level Implementation with `nn.Module`

In [135]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # 这里的4代表的是输入的参数为4，例如：可以输入torch.FloatTensor([1, 2, 3, 4])

    def forward(self, x):
        return self.linear(x)

In [162]:
model = SoftmaxClassifierModel()
print(model.parameters())
for ii in model.parameters():
    print(ii)

<generator object Module.parameters at 0x000001BF13045748>
Parameter containing:
tensor([[-0.3932, -0.3522,  0.2444, -0.3592],
        [-0.1146,  0.3637,  0.3960,  0.4729],
        [-0.1015, -0.3886,  0.4923, -0.1065]], requires_grad=True)
Parameter containing:
tensor([-0.2057,  0.1219, -0.3497], requires_grad=True)


Let's try another new dataset.

In [163]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1) # 给到 W和b

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 3.922793
Epoch  100/1000 Cost: 0.702366
Epoch  200/1000 Cost: 0.618464
Epoch  300/1000 Cost: 0.560044
Epoch  400/1000 Cost: 0.508856
Epoch  500/1000 Cost: 0.460656
Epoch  600/1000 Cost: 0.413828
Epoch  700/1000 Cost: 0.367608
Epoch  800/1000 Cost: 0.321795
Epoch  900/1000 Cost: 0.277590
Epoch 1000/1000 Cost: 0.245153


In [164]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1) # 给到 W和b


# prediction = model(x_train)
# print("prediction",prediction)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    prediction = model(x_train)

    hypothesis = F.softmax(prediction,dim = 1)

    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)  # 在第 0 维上填充值

    cost = (-y_one_hot * torch.log(hypothesis)).sum(dim=1).mean()

    # 梯度清零，反向传播，参数更新
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 每 100 次迭代输出一次日志
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

    

Epoch    0/1000 Cost: 0.244932
Epoch  100/1000 Cost: 0.231630
Epoch  200/1000 Cost: 0.220408
Epoch  300/1000 Cost: 0.210192
Epoch  400/1000 Cost: 0.200851
Epoch  500/1000 Cost: 0.192277
Epoch  600/1000 Cost: 0.184380
Epoch  700/1000 Cost: 0.177083
Epoch  800/1000 Cost: 0.170320
Epoch  900/1000 Cost: 0.164038
Epoch 1000/1000 Cost: 0.158186
